In [ ]:
from pathlib import Path
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans

from cloud_io.gcp.io import download_file
from kmeans.tools.common_path import DATA_PATH, BUCKET_NAME, ROOT_PATH

In [ ]:
FIGSIZE = (15, 5)

# Collecte de données

Le TD est basé sur le notebook Kaggle https://www.kaggle.com/code/shrutimechlearn/step-by-step-kmeans-explained-in-detail/notebook

In [ ]:
download_path = Path(DATA_PATH, "Mall_Customers.csv") 
path_data = download_file(download_path, bucket_as_local=ROOT_PATH, bucket_name=BUCKET_NAME)

In [ ]:
df = pd.read_csv(path_data,index_col='CustomerID')
df.head()

## Compréhension des variables

La base de donnée provient d'un centre commercial (ou magasin) qui recense ses clients:

* CustomerID: ID du client
* Genre: Homme (Male) et Femme (Female)
* Age: âge du client
* Annual_Income_(k$): Revenu annuel en millier de dollars du client
* Spending_Score: variable (entre 1-100) synthétisant le comportement du client, plus il dépense fréquemment plus le score sera élevé.

In [ ]:
 # description des variables numériques

# Problème à résoudre 

Comment varie le score de dépense (Spending_Score) avec le revenu annuel (Annual_Income)?

In [ ]:
df

In [ ]:
# scatterplot y= 'Spending_Score' et x = 'Annual_Income_(k$)' avec des couleurs différente par genre


# Implémentation de l'algorithme

## Quel devrait être le nombre de cluster?

In [ ]:
X = df[['Annual_Income_(k$)','Spending_Score']].copy()

In [ ]:
# Using the elbow method to find the optimal number of clusters
wcss = []
for i in range(1, 11):
    modele_kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    modele_kmeans.fit(X)
    # inertia method returns wcss for that model
    wcss.append(modele_kmeans.inertia_)

**wcss**: Within Cluster Sum of Squares. il s'agit donc de l'Inertie intraclasse

In [ ]:
plt.figure(figsize = FIGSIZE)
sns.lineplot(x = range(1, 11), y = wcss,marker='o',color='red')
plt.title('La méthode du coude')
plt.xlabel('Nombre de classe')
plt.ylabel('')
plt.show()

On choisit 5 classes

In [ ]:
# Fitting K-Means to the dataset
modele_kmeans = KMeans(n_clusters = 5, init = 'k-means++', random_state = 42)
y_kmeans = modele_kmeans.fit_predict(X)

In [ ]:
X["cluster"] = y_kmeans
centroides = pd.DataFrame(modele_kmeans.cluster_centers_)

## Visualisation du résultat 

In [ ]:
# Visualising the clusters
plt.figure(figsize=FIGSIZE)
sns.scatterplot(data=X,  x='Annual_Income_(k$)', y='Spending_Score', hue='cluster', palette="Paired", s=50)
sns.scatterplot(data=centroides, x=0, y=1, color='red', label='Centroides', s=300, marker=',')
plt.title('Clusters of customers')
plt.xlabel('Annual Income (k$)')
plt.ylabel('Spending Score (1-100)')
plt.legend()
plt.show()

## Essayons de comprendre ces classes

Renommons les classes avec ce que nous remarquons:

* Classe 0: XXXX0
* Classe 1: XXXX1
* Classe 2: XXXX2
* Classe 3: XXXX3
* Classe 4: XXXX4

In [ ]:
df["cluster"] = y_kmeans
df['cluster_label'] = np.where(df['cluster'] == 0, 'XXXX0',
                               np.where(df['cluster'] == 1, 'XXXX1',
                                        np.where(df['cluster'] == 2, 'XXXX2', 
                                                 np.where(df['cluster'] == 3, 'XXXX3',
                                                         'XXXX4'))))

In [ ]:
df.groupby("cluster_label").agg("mean")

**Conclusion**
Il y a des dépensiers dans les revenus extrêmes (revenu élevé et revenu). Les dépensiers semblent être identifiés par leur âge.

# **FIN DU NOTEBOOK**